# 🚀 Credit Card Fraud Detection Pipeline

This notebook demonstrates **EDA, Anomaly Detection, Machine Learning Models, and Real-time Simulation** for fraud detection using the credit card dataset.

## 📥 1. Load Data Efficiently

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load dataset
data = pd.read_csv("creditcard.csv")

print("Shape of dataset:", data.shape)
print(data.head())

## 📊 2. Basic EDA

In [ ]:
print("\nClass distribution:")
print(data["Class"].value_counts())

plt.figure(figsize=(6,4))
sns.countplot(x='Class', data=data)
plt.title("Fraud (1) vs Normal (0)")
plt.show()

# Correlation Heatmap
plt.figure(figsize=(12,6))
sns.heatmap(data.corr(), cmap="coolwarm", cbar=False)
plt.title("Feature Correlation")
plt.show()

## 🔍 3. Anomaly Detection

In [ ]:
from sklearn.ensemble import IsolationForest

iso = IsolationForest(contamination=0.001, random_state=42)
data['anomaly'] = iso.fit_predict(data.drop(['Class'], axis=1))

print("\nAnomaly Detection Results:")
print(data['anomaly'].value_counts())

## ✂️ 4. Train-Test Split + Sampling

In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop(["Class", "anomaly"], axis=1)
y = data["Class"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## ⚖️ 5. Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## 🤖 6. ML Models
We train **Logistic Regression, Decision Tree, and Neural Network**.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, class_weight='balanced'),
    "Decision Tree": DecisionTreeClassifier(class_weight='balanced'),
    "Neural Net": MLPClassifier(hidden_layer_sizes=(64,32), max_iter=50)
}

# Store performance
performance = []

for name, model in models.items():
    print(f"\n===== {name} =====")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))
    
    # Collect metrics
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
    performance.append([name, precision, recall, f1])
    
    # Confusion Matrix
    sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
    plt.title(f"Confusion Matrix - {name}")
    plt.show()

# Create summary table
perf_df = pd.DataFrame(performance, columns=["Model", "Precision", "Recall", "F1-Score"])
print("\n📊 Model Performance Summary:")
print(perf_df)

## ⚡ 7. Real-time Monitoring Simulation

In [ ]:
print("\nSimulating Real-Time Fraud Detection:")
for i in range(5):
    sample = X_test[i].reshape(1, -1)
    pred = models["Logistic Regression"].predict(sample)
    print(f"Transaction {i+1}: Fraud" if pred==1 else f"Transaction {i+1}: Normal")

## 📈 8. Scalability Tip

In [ ]:
# In production -> use streaming frameworks (Kafka, Spark) & incremental learning